In [ ]:
import sys
sys.path.insert(0, '../')

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from matplotlib.patches import Patch

from utils.data_utils import BratsDataset3D
from utils.predict import ModelPredict

In [ ]:
model_path = '../model.bin'
model = ModelPredict(model_path)

In [ ]:
scan_dir = r'/Users/colinodowd/Downloads/nii/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
scan_type = 'flair'
scan_index = 0

[input_tensor, label_tensor] = BratsDataset3D(scan_dir, scan_type).__getitem__(scan_index)

model_output = model.predict(input_tensor)
pred_np = model_output.squeeze().numpy()  
label_np = label_tensor.squeeze().numpy()  

In [ ]:
scan_data = model.get_scan()

num_slices = 5
middle_slice = 75  # Adjust as needed
num_rows = 1
num_cols = num_slices

plt.figure(figsize=(20, 5))

for i in range(num_slices):
    current_slice = middle_slice + 3 * i
        
    scan_slice = scan_data[:, :, current_slice]
    pred_slice = pred_np[:, :, current_slice]
    actual_slice = label_np[:, :, current_slice]
    # TODO
    plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(pred_np[:, :, middle_slice + 3 * i], cmap='gray') 
    # TODO
    plt.title('Middle Slice + {}'.format(3 * i))
    plt.axis('off')
plt.tight_layout()
plt.show()